# COGS 189: Final Project
### Members: Ehson Pirouzian, ... write your own

This is our Winter 2022 Final Project for COGS 189
The purpose is a machine learning project that relates to motor sensory data from BCI competition IV dataset 2a

### Hypothesis

Is it easiest to classify data left hand sensory data against right hand, foot, or tounge sensory data?

### Code

We will use matlab and python for how we test our hypothesis


#### Importing the data

In [76]:
import os
import numpy as np
import pandas as pd
from pprint import pprint
import scipy.io as sio
import mne

def get_data(subject,training,PATH):
	NO_channels = 22
	NO_tests = 6*48 	
	Window_Length = 7*250 

	class_return = np.zeros(NO_tests)
	data_return = np.zeros((NO_tests,NO_channels,Window_Length))

	NO_valid_trial = 0
	if training:
		a = sio.loadmat(PATH+'A0'+str(subject)+'T.mat')
	else:
		a = sio.loadmat(PATH+'A0'+str(subject)+'E.mat')
	a_data = a['data']
	for ii in range(0,a_data.size):
		a_data1 = a_data[0,ii]
		a_data2=[a_data1[0,0]]
		a_data3=a_data2[0]
		a_X 		= a_data3[0]
		a_trial 	= a_data3[1]
		a_y 		= a_data3[2]
		a_fs 		= a_data3[3]
		a_classes 	= a_data3[4]
		a_artifacts = a_data3[5]
		a_gender 	= a_data3[6]
		a_age 		= a_data3[7]
		for trial in range(0,a_trial.size):
			if(a_artifacts[trial]==0):
				data_return[NO_valid_trial,:,:] = np.transpose(a_X[int(a_trial[trial]):(int(a_trial[trial])+Window_Length),:22])
				class_return[NO_valid_trial] = int(a_y[trial])
				NO_valid_trial +=1


	return data_return[0:NO_valid_trial,:,:], class_return[0:NO_valid_trial]
	

# Now we will import each of the 9 training files
df1, classes1 = get_data("1",True,"./dataset2a/")
df2, classes2 = get_data("2",True,"./dataset2a/")
df3, classes3 = get_data("3",True,"./dataset2a/")
df4, classes4 = get_data("4",True,"./dataset2a/")
df5, classes5 = get_data("5",True,"./dataset2a/")
df6, classes6 = get_data("6",True,"./dataset2a/")
df7, classes7 = get_data("7",True,"./dataset2a/")
# df8, classes8 = get_data("8",True,"./dataset2a/")
# df9, classes9 = get_data("9",True,"./dataset2a/")


In [102]:
# 4 classes, #some number of tests, 3 channels, 1750 datapoints each
# this will become 4 different dataframes for each class with 3 channels and #tests for each channel * datapoints

leftHandData = pd.DataFrame(columns=['EEGC3', 'EEGCz', 'EEGC4'])
rightHandData = pd.DataFrame(columns=['EEGC3', 'EEGCz', 'EEGC4'])
footData = pd.DataFrame(columns=['EEGC3', 'EEGCz', 'EEGC4'])
toungeData = pd.DataFrame(columns=['EEGC3', 'EEGCz', 'EEGC4'])
index = 0

### this is horrid. it is litterally gonna crash your PC

for val in classes1:
    if(int(val) == 1):
        for dataIndex in range(df1[index][7].size):
            if(dataIndex > 750):
                leftHandData.append({'EEGC3': df1[index][7][dataIndex]},ignore_index=True)
                leftHandData.append({'EEGCz': df1[index][9][dataIndex]},ignore_index=True)
                leftHandData.append({'EEGC4': df1[index][11][dataIndex]},ignore_index=True)
    index = index + 1


    




(262, 22, 1750)
(262,)


AttributeError: 'DataFrame' object has no attribute 'concat'

Here I will explain our reasoning for the above code






As a citation of sources we used this data set as a guide for formatting our data

1. https://github.com/MultiScale-BCI/IV-2a/blob/master/  by "Michael Hersche and Tino Rellstab"